In [2]:
############################################################
#  Program Settings
############################################################
%tensorflow_version 1.5.0
import time
import numpy as np
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/Colab Notebooks/')
sys.path.append('/content/drive/My Drive/Colab Notebooks/mcrnn/')
import os
import pickle
from keras.models import load_model
import matplotlib.pyplot as plt
from mrcnn.config import Config
from mrcnn import visualize
from mrcnn import model as modellib, utils
from mrcnn.visualize import display_images
import cv2
from google.colab.patches import cv2_imshow
!nvidia-smi
############################################################
#  Configurations
############################################################

# A config for both in case we eventually use one differently than the other
class FullPalletConfig(Config):

    NAME = 'full-pallet'
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1 
    NUM_CLASSES = 2  # Background + object of interest
    DETECTION_MIN_CONFIDENCE = 0.965
    # Some stuff to speed it up, comment out if necessary
    # MAX_GT_INSTANCES = 100
    # IMAGE_MIN_DIM = 512
    # IMAGE_MAX_DIM = 512
    # TRAIN_ROIS_PER_IMAGE = 200

class ProdOnPalletConfig(Config):
    NAME = 'product-on-pallet'
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 1  # Background + object of interest
    DETECTION_MIN_CONFIDENCE = 0.965

############################################################
#  Model stuff
############################################################
# For writing on the image
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 0.75
fontColor1 = (0, 0, 0)
fontColor2 = (255, 255, 255)
lineType = 1

print("++++++++++++++\nLOADING MODELS\n++++++++++++++\n")
models_dir = '/content/drive/My Drive/Colab Notebooks/mrcnn/'
full_pallet_mod_name = 'Copy of mask_rcnn_full-pallet_0181.h5'
product_mod_name = 'Copy of mask_rcnn_product-on-pallet_0002.h5'

pallet_model =  modellib.MaskRCNN(mode="inference", config=FullPalletConfig(), model_dir='/content/drive/My Drive/Colab Notebooks/full-pallet/logs')
product_model =  modellib.MaskRCNN(mode="inference", config=ProdOnPalletConfig(), model_dir='/content/drive/My Drive/Colab Notebooks/full-pallet/logs')

pallet_model.load_weights(models_dir + full_pallet_mod_name, by_name=True)
product_model.load_weights(models_dir + product_mod_name, by_name=True)

in_path = '/content/drive/My Drive/Colab Notebooks/test_common/'
out_path = '/content/drive/My Drive/Colab Notebooks/out_common/'
num_images = len(os.listdir(in_path))
n = 1

print('Starting...\n')
start_time = time.time()
img_data = {}

max_time = 0 
avg_time = 0
total_time = 0
for f in os.listdir(in_path):
  print(f)
  im_start_time = time.time()
  print(n, '/', num_images)

  # First load the image
  image = cv2.imread(in_path + f)
  # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  # Look for a product in the image
  products = product_model.detect([image], verbose=False)
  r_products = products[0]
  products_boxes = r_products['rois']

  # If product is found, look for a pallet in the image
  pallet_boxes = []
  if len(products_boxes) > 0:

    # Find a pallet/pallets
    pallets = pallet_model.detect([image], verbose=False)
    r_pallets = pallets[0]
    pallet_boxes = r_pallets['rois']
      
  # If a pallet is found, use one of them to infer the height in pixels of the product found
  if len(pallet_boxes) > 0:
    print(len(pallet_boxes))
    """
    Method: find the width of the pallet at the midpoint of the bounding box
    and use this measurement to infer the height. For now we'll just use one 
    pallet and one bounding box even if multiple are found and see how that 
    goes
    """
    rect = pallet_boxes[0,:] # Just take the first one found for this version
    rect_width = rect[3] - rect[1]
    rect_height = rect[2] - rect[0]
    rect_midpt_pixel = rect[1] + int(rect_width/2)

    """
    Now that we have the pixel in the image that represents the midpt of
    the bounding box wrt the edge of the image, let's look at that row in
    the mask within the bounding box
    """
    pallet_mask = r_pallets['masks'][rect[0]:rect[2], rect_midpt_pixel, 0].astype(int)
    six_inches = sum(pallet_mask)  # the pallet is this many pixels high (6")

    """
    Now that we have the number of pixels equal to 6 inches, find the heights of the products.
    We'll measure from these two points:
    bottom: the point where the mask starts at the midpt of the bounding box, minus the pallet
    top: the top of the bounding box
    """
    mask_idx=0
    p = 1
    for prod in products_boxes:
      p+=1
      # midpt pixel
      prod_width = prod[3] - prod[1]
      prod_midpt_pixel = prod[1] + int(prod_width/2)

      # Find where the prorudct mask begins
      product_mask = r_products['masks'][:, prod_midpt_pixel, mask_idx].astype(int)
      mask_idx += 1
      pts = np.where(product_mask != 0)
      bottom_pt = np.max(pts)

      # Get the product height first
      product_height = bottom_pt - prod[0]

      # subtract the pallet (bounding box)
      product_height -= six_inches 

      # translate from pixels to inches
      product_height_inches = int((product_height/six_inches)*6)

      ft = int(product_height_inches/12)
      inches = product_height_inches%12

      # Draw an arrow in the bounding box:
      midpt = int((prod[3]-prod[1])/2)
      pt1 = (prod_midpt_pixel, prod[0])
      pt2 = (prod_midpt_pixel, bottom_pt-six_inches)
      image = cv2.arrowedLine(image, pt1, pt2, (0,255,0), 2)
      image = cv2.arrowedLine(image, pt2, pt1, (0,255,0), 2)

      # Black border, white font
      cv2.putText(image, str(ft)+'\''+str(inches)+'\"',
                  (prod_midpt_pixel+4, int(prod[2] - rect_height - (product_height/2))),
                  fontFace=cv2.FONT_HERSHEY_SIMPLEX , fontScale=0.8, color=[0, 0, 0], lineType=1, thickness=4)
      cv2.putText(image, str(ft)+'\''+str(inches)+'\"',
                  (prod_midpt_pixel+4, int(prod[2] - rect_height - (product_height/2))),
                  fontFace=cv2.FONT_HERSHEY_SIMPLEX , fontScale=0.8, color=[255, 255, 255], lineType=1, thickness=2)

          
      # No need to save for timing 
    print("Saving")
    cv2.imwrite(out_path + f, image)
      # img_data[str(f)] = str(product_height_inches)
  
  im_time = time.time() - im_start_time
  if im_time > max_time:
    max_time = im_time
    max_time_pic = n
  total_time += im_time
  avg_time = total_time/n
  print(f, '|', avg_time, '|', im_time, '|', max_time, '|', max_time_pic, '|', len(pallet_boxes), '|', len(products_boxes) )
  n += 1

  # with open('/content/drive/My Drive/Colab Notebooks/data.txt', 'w') as fname:
  #   fname.write(str(img_data))

print('takes', time.time()-start_time, 'to process', num_images, 'images')

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.5.0`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Tue May  5 03:05:23 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    26W /  75W |   5509MiB /  7611MiB |      0%      Default |